In [2]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("Combined_Sector_DataFrame_with_Sector_Column.csv")
df.drop('DX113', inplace=True, axis=1)

In [7]:
df.head()

,Securities,BI001,ES106,ES105,F0946,ES076,ES077,ES015,ES014,ES494,...,RR251,RR472,IS010,RR861,RR009,RR530,IS147,CF015,RR014,RR008
0,Security_Communications_1,Communications,71.480127,55.518686,67.088116,34.442149,31.096492,76.950809,88.241015,66.071563,...,49.138937,44.221886,92.270062,91.287572,56.125416,48.902691,29.182852,71.764348,19.235700,6.637985
1,Security_Communications_2,Communications,61.222378,61.711447,29.550310,6.097849,38.899276,65.120920,45.166419,90.849462,...,31.808868,34.946370,70.314391,10.590859,30.757697,25.849600,45.729597,52.413377,49.573710,69.084310
2,Security_Consumer Discretionary_1,Consumer Discretionary,43.998717,47.753453,63.347476,14.962167,55.745549,55.578844,29.223364,2.478955,...,19.537116,90.927053,84.233647,1.850779,25.345150,29.955286,40.209436,5.824926,95.049755,19.088272
3,Security_Consumer Discretionary_2,Consumer Discretionary,90.771757,66.208428,64.005245,61.605830,29.675923,68.671957,6.763775,97.426893,...,29.112433,99.680992,96.974329,4.611684,84.427190,61.234389,32.113075,6.475916,54.357004,51.573768
4,Security_Consumer Staples_1,Consumer Staples,44.417033,68.160743,10.754324,91.584387,58.010964,74.470258,90.935959,46.301274,...,45.651791,17.888520,61.262138,40.060699,39.260963,17.070454,64.618259,61.307263,79.233245,69.558122


In [37]:
df = df.iloc[:, :18]

In [8]:
df.DX113.unique()

AttributeError: 'DataFrame' object has no attribute 'DX113'

In [56]:
df = df[df.DX113 =='JP']

In [9]:
df.shape

(22, 37)

In [10]:
df.BI001.unique()
print(len(df.BI001.unique()))

11


In [11]:
df.BI001.unique()

array(['Communications', 'Consumer Discretionary', 'Consumer Staples',
       'Energy', 'Financials', 'Health Care', 'Industrials', 'Materials',
       'Real Estate', 'Technology', 'Utilities'], dtype=object)

In [38]:
dfs = []

sectors = sorted(df.BI001.unique())

for i in sectors:
    temp = df[df.BI001 == i]
    dfs.append(temp)

In [39]:
print(dfs[0].columns)
dfs[0].head()

Index(['Securities', 'BI001', 'ES106', 'ES105', 'F0946', 'ES076', 'ES077',
       'ES015', 'ES014', 'ES494', 'ES020', 'ES021', 'ES043', 'ES047', 'ES063',
       'ES096', 'ES319', 'SA848'],
      dtype='object')


,Securities,BI001,ES106,ES105,F0946,ES076,ES077,ES015,ES014,ES494,ES020,ES021,ES043,ES047,ES063,ES096,ES319,SA848
0,Security_Communications_1,Communications,71.480127,55.518686,67.088116,34.442149,31.096492,76.950809,88.241015,66.071563,43.891354,88.112709,73.210241,58.259170,91.627225,83.701345,38.691802,62.729793
1,Security_Communications_2,Communications,61.222378,61.711447,29.550310,6.097849,38.899276,65.120920,45.166419,90.849462,81.785205,3.506379,16.300096,52.619277,72.372433,90.968984,58.886008,7.180285


# Model Structure

Get score -> get weights -> weight the scores -> get E, S, G -> get ESG score

first i will just try to get the score for communications sector

In [40]:
communications = dfs[0]

In [41]:
sectors

['Communications',
 'Consumer Discretionary',
 'Consumer Staples',
 'Energy',
 'Financials',
 'Health Care',
 'Industrials',
 'Materials',
 'Real Estate',
 'Technology',
 'Utilities']

### Get Scores

In [42]:
# scores are determined based on the quantiles within each factor against the sector

def score_by_quantile(series):
    return pd.qcut(series, q=4, labels=[0, 1, 2, 3])

# Create a copy of the DataFrame to store the scores
df_scores = df.copy()

In [63]:
# Apply the scoring for each factor column
for col in df_scores.columns:
    # if columns are score related
    if col != 'Securities' and col != 'BI001':
        df_scores[col] = score_by_quantile(df[col])
        df_scores[col] = df_scores[col].astype(int)

In [64]:
df_scores

,Securities,BI001,ES106,ES105,F0946,ES076,ES077,ES015,ES014,ES494,ES020,ES021,ES043,ES047,ES063,ES096,ES319,SA848
0,Security_Communications_1,Communications,2,1,3,1,1,3,3,2,1,3,2,1,3,3,1,1
1,Security_Communications_2,Communications,2,2,1,0,1,2,1,3,3,0,0,1,2,3,2,0
2,Security_Consumer Discretionary_1,Consumer Discretionary,1,1,2,0,2,1,0,0,3,0,1,1,3,0,3,2
3,Security_Consumer Discretionary_2,Consumer Discretionary,3,2,2,3,0,2,0,3,3,2,2,3,1,3,2,2
4,Security_Consumer Staples_1,Consumer Staples,1,3,0,3,2,3,3,1,0,0,0,2,1,0,1,0
5,Security_Consumer Staples_2,Consumer Staples,3,3,2,1,3,1,3,0,2,1,1,0,1,2,0,2
6,Security_Energy_1,Energy,0,1,2,3,1,0,3,3,2,1,2,3,2,2,3,3
7,Security_Energy_2,Energy,1,0,1,2,3,3,1,1,0,1,1,3,2,0,2,1
8,Security_Financials_1,Financials,3,0,2,3,0,0,1,0,2,0,0,3,0,2,2,2
9,Security_Financials_2,Financials,3,2,0,1,0,1,2,0,0,2,0,2,1,0,2,3


### Get Weights

In [65]:
# assume that feature selections and eng is done

In [66]:
# weights 

E_weight = 0.3
S_weight = 0.3
G_weight = 0.3
completion_bonus_weight = 0.1


# factor weights **this differs by sector**
factors_name = df_scores.iloc[:, 2:].columns

In [67]:
print(factors_name)
len(factors_name)

Index(['ES106', 'ES105', 'F0946', 'ES076', 'ES077', 'ES015', 'ES014', 'ES494',
       'ES020', 'ES021', 'ES043', 'ES047', 'ES063', 'ES096', 'ES319', 'SA848'],
      dtype='object')


16

In [68]:
factors_weights = np.random.rand(len(factors_name))  # This is what we need to determine
weights_dict = dict(zip(factors_name, factors_weights))

weights  = [weights_dict.get(key) for key in factors_name]

### Weights Calculation

In [69]:
for factor in factors_name:
    df_scores[factor] = df_scores[factor] * weights_dict.get(factor)

### Score Calculation

In [70]:

# Environmentals
E_total = df_scores.loc[:, 'ES106':'ES021'].sum(axis=1)
S_total = df_scores.loc[:,"ES043":"ES047"].sum(axis=1)
G_total = df_scores.loc[:,"ES063":"SA848"].sum(axis=1)


# rewards for data completeness based on the number of non-null values
Completion_bonus_total = 0

ESG_Score = E_total*E_weight + S_total*S_weight + G_total*G_weight

# create dataframe to store the score - securities, E, S, G, ESG
Scores = pd.DataFrame()
Scores['Securities'] = df_scores['Securities']
Scores['E'] = E_total
Scores['S'] = S_total
Scores['G'] = G_total
Scores['ESG'] = ESG_Score
# also add sector name

In [71]:
Scores

,Securities,E,S,G,ESG
0,Security_Communications_1,8.931985,1.869576,4.958740,4.728090
1,Security_Communications_2,4.640974,0.854683,5.210909,3.211970
2,Security_Consumer Discretionary_1,3.513170,1.362129,4.264761,2.742018
3,Security_Consumer Discretionary_2,6.733266,3.578941,5.344414,4.696986
4,Security_Consumer Staples_1,6.402155,1.709365,1.252595,2.809235
5,Security_Consumer Staples_2,9.082911,0.507447,2.684256,3.682384
6,Security_Energy_1,7.203166,3.578941,5.948591,5.019209
7,Security_Energy_2,5.094522,3.071494,2.758678,3.277408
8,Security_Financials_1,5.419357,2.564048,4.069038,3.615733
9,Security_Financials_2,5.375439,1.709365,2.892183,2.993096


**possible problem**
 
if we change the weights for each sector, score are only comparable within the sector
so you cannot really get cross-sector insights. This might be critical for sellside equity analyst

However...
if we use same weights across sectors, then we can compare the scores but sectors that are "easy ESG" will have higher scores.

### Run loop to get scores for all sectors

In [76]:
df = pd.read_csv("Combined_Sector_DataFrame_with_Sector_Column.csv")
df = df.iloc[:, :18] # this is to filter the cols we need (remove traditional FA cols)

# df = df[df.DX113 =='JP']
# df.drop('DX113', inplace=True, axis=1)
df.head()

,Securities,BI001,ES106,ES105,F0946,ES076,ES077,ES015,ES014,ES494,ES020,ES021,ES043,ES047,ES063,ES096,ES319,SA848
0,Security_Communications_1,Communications,71.480127,55.518686,67.088116,34.442149,31.096492,76.950809,88.241015,66.071563,43.891354,88.112709,73.210241,58.259170,91.627225,83.701345,38.691802,62.729793
1,Security_Communications_2,Communications,61.222378,61.711447,29.550310,6.097849,38.899276,65.120920,45.166419,90.849462,81.785205,3.506379,16.300096,52.619277,72.372433,90.968984,58.886008,7.180285
2,Security_Consumer Discretionary_1,Consumer Discretionary,43.998717,47.753453,63.347476,14.962167,55.745549,55.578844,29.223364,2.478955,85.659521,3.050136,53.432313,46.082983,87.030940,21.476085,96.776475,76.486982
3,Security_Consumer Discretionary_2,Consumer Discretionary,90.771757,66.208428,64.005245,61.605830,29.675923,68.671957,6.763775,97.426893,98.601064,48.052798,67.061919,95.552016,40.724389,95.183424,52.510143,88.945313
4,Security_Consumer Staples_1,Consumer Staples,44.417033,68.160743,10.754324,91.584387,58.010964,74.470258,90.935959,46.301274,16.227657,0.553029,6.715614,71.421453,52.329440,15.815864,28.658556,49.097177


In [77]:

dfs = []

sectors = sorted(df.BI001.unique())

for i in sectors:
    temp = df[df.BI001 == i]
    dfs.append(temp)

In [78]:

# Initialize an empty DataFrame to store the combined sector scores
combined_scores = pd.DataFrame()

In [81]:
def calculate_esg_scores(df, factors_name, weights_dict, E_weight=0.3, S_weight=0.3, G_weight=0.3, completion_bonus_weight=0.1):
    """
    Calculate the ESG scores for a given DataFrame using predefined factor weights.
    
    Parameters:
    df: DataFrame
        DataFrame containing securities and ESG factors.
    factors_name: list
        List of ESG factor column names.
    weights_dict: dict
        Dictionary mapping ESG factors to their respective weights.
    E_weight: float, optional
        Weight for Environmental score (default is 0.3).
    S_weight: float, optional
        Weight for Social score (default is 0.3).
    G_weight: float, optional
        Weight for Governance score (default is 0.3).
    completion_bonus_weight: float, optional
        Weight for completion bonus score (default is 0.1).
    
    Returns:
    Scores: DataFrame
        DataFrame containing calculated E, S, G, and ESG scores for each security.
    """

    # Helper function to assign quantile-based scores
    def score_by_quantile(series):
        return pd.qcut(series, q=4, labels=[0, 1, 2, 3])

    # Create a copy of the DataFrame to store the scores
    df_scores = df.copy()

    # Apply the quantile-based scoring for each factor column
    for col in factors_name:
        if col != 'Securities' and col != 'BI001':
            df_scores[col] = score_by_quantile(df[col])
            df_scores[col] = df_scores[col].astype(int)

    # Multiply each factor's score by its corresponding weight
    for factor in factors_name:
        df_scores[factor] = df_scores[factor] * weights_dict.get(factor)

    # Calculate the total scores for Environmental, Social, and Governance factors
    E_total = df_scores.loc[:, 'ES106':'ES021'].sum(axis=1)
    S_total = df_scores.loc[:, "ES043":"ES047"].sum(axis=1)
    G_total = df_scores.loc[:, "ES063":"SA848"].sum(axis=1)

    # Calculate rewards for data completeness (Completion bonus)
    Completion_bonus_total = df_scores[factors_name].notna().sum(axis=1) * completion_bonus_weight

    # Calculate the final ESG score based on the E, S, G scores and completion bonus
    ESG_Score = E_total * E_weight + S_total * S_weight + G_total * G_weight + Completion_bonus_total

    # Create a DataFrame to store the final scores
    Scores = pd.DataFrame()
    Scores['Securities'] = df_scores['Securities']
    Scores['E'] = E_total
    Scores['S'] = S_total
    Scores['G'] = G_total
    Scores['ESG'] = ESG_Score

    return Scores

In [82]:
factors_name = df_scores.iloc[:, 2:].columns
factors_weights = np.random.rand(len(factors_name))  # This is what we need to determine
weights_dict = dict(zip(factors_name, factors_weights))

In [83]:
# Loop through each sector and its corresponding DataFrame
for sector, df_sector in zip(sectors, dfs):
    # Calculate ESG scores for the current sector
    sector_scores = calculate_esg_scores(df_sector, factors_name, weights_dict)

    # Add a column for the sector name
    sector_scores['Sector'] = sector

    # Append the current sector's scores to the combined DataFrame
    combined_scores = pd.concat([combined_scores, sector_scores], ignore_index=True)

In [84]:
combined_scores

,Securities,E,S,G,ESG,Sector
0,Security_Communications_1,11.185683,3.782567,2.494103,6.838706,Communications
1,Security_Communications_2,8.724526,0.000000,1.434816,4.647802,Communications
2,Security_Consumer Discretionary_1,3.835334,0.000000,2.612200,3.534260,Consumer Discretionary
3,Security_Consumer Discretionary_2,16.074874,3.782567,1.316718,7.952248,Consumer Discretionary
4,Security_Consumer Staples_1,7.487218,1.499577,2.612200,5.079699,Consumer Staples
5,Security_Consumer Staples_2,12.422990,2.282990,1.316718,6.406809,Consumer Staples
6,Security_Energy_1,11.260975,2.282990,3.928918,6.841865,Energy
7,Security_Energy_2,8.649233,1.499577,0.000000,4.644643,Energy
8,Security_Financials_1,11.755698,1.499577,0.550502,5.741733,Financials
9,Security_Financials_2,8.154510,2.282990,3.378417,5.744775,Financials
